In [1]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/matching_engine/matching_engine_for_indexing.ipynb">
      Run in Google Cloud Notebooks
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/matching_engine/matching_engine_for_indexing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

This example demonstrates how to use the GCP ANN Service. It is a high scale, low latency solution, to find similar vectors (or more specifically "embeddings") for a large corpus. Moreover, it is a fully managed offering, further reducing operational overhead. It is built upon [Approximate Nearest Neighbor (ANN) technology](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) developed by Google Research.

### Dataset

The dataset used for this tutorial is the [GloVe dataset](https://nlp.stanford.edu/projects/glove/).

### Objective

In this notebook, you will learn how to create Approximate Nearest Neighbor (ANN) Index, query against indexes, and validate the performance of the index. 

The steps performed include:

* Create ANN Index and Brute Force Index
* Create an IndexEndpoint with VPC Network
* Deploy ANN Index and Brute Force Index
* Perform online query
* Compute recall


### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

* **Prepare a VPC network**.  To reduce any network overhead that might lead to unnecessary increase in overhead latency, it is best to call the ANN endpoints from your VPC via a direct [VPC Peering](https://cloud.google.com/vertex-ai/docs/general/vpc-peering) connection. The following section describes how to setup a VPC Peering connection if you don't have one. This is a one-time initial setup task. You can also reuse existing VPC network and skip this section.
* **WARNING:** The match service gRPC API (to create online queries against your deployed index) has to be executed in a Google Cloud Notebook instance that is created with the following requirements:
  * **In the same region as where your ANN service is deployed** (for example, if you set `REGION = "us-central1"` as same as the tutorial, the notebook instance has to be in `us-central1`).
  * **Make sure you select the VPC network you created for ANN service** (instead of using the "default" one). That is, you will have to create the VPC network below and then create a new notebook instance that uses that VPC.  
  * If you run it in the colab or a Google Cloud Notebook instance in a different VPC network or region, the gRPC API will fail to peer the network (InactiveRPCError).

In [12]:
PROJECT_ID = 'rec-ai-demo-326116'  # @param {type:"string"}
NETWORK_NAME = "default"  # @param {type:"string"}
PEERING_RANGE_NAME = 'google-reserved-range'
BUCKET = 'rec_bq_jsw' # Change to the bucket you created.

# Create a VPC network
! gcloud compute networks create {NETWORK_NAME} --bgp-routing-mode=regional --subnet-mode=auto --project={PROJECT_ID}

# Add necessary firewall rules
! gcloud compute firewall-rules create {NETWORK_NAME}-allow-icmp --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow icmp

! gcloud compute firewall-rules create {NETWORK_NAME}-allow-internal --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow all --source-ranges 10.128.0.0/9

! gcloud compute firewall-rules create {NETWORK_NAME}-allow-rdp --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow tcp:3389

! gcloud compute firewall-rules create {NETWORK_NAME}-allow-ssh --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow tcp:22

# Reserve IP range
! gcloud compute addresses create {PEERING_RANGE_NAME} --global --prefix-length=16 --network={NETWORK_NAME} --purpose=VPC_PEERING --project={PROJECT_ID} --description="peering range for uCAIP Haystack."

# Set up peering with service networking
! gcloud services vpc-peerings connect --service=servicenetworking.googleapis.com --network={NETWORK_NAME} --ranges={PEERING_RANGE_NAME} --project={PROJECT_ID}

ERROR: (gcloud.compute.networks.create) Could not fetch resource:
 - The resource 'projects/rec-ai-demo-326116/regions/us-central1/subnetworks/default' already exists

Creating firewall...⠹Created [https://www.googleapis.com/compute/v1/projects/rec-ai-demo-326116/global/firewalls/default-allow-icmp].
Creating firewall...done.                                                      
NAME                NETWORK  DIRECTION  PRIORITY  ALLOW  DENY  DISABLED
default-allow-icmp  default  INGRESS    65534     icmp         False
Creating firewall...⠹Created [https://www.googleapis.com/compute/v1/projects/rec-ai-demo-326116/global/firewalls/default-allow-internal].
Creating firewall...done.                                                      
NAME                    NETWORK  DIRECTION  PRIORITY  ALLOW  DENY  DISABLED
default-allow-internal  default  INGRESS    65534     all          False
Creating firewall...⠹Created [https://www.googleapis.com/compute/v1/projects/rec-ai-demo-326116/global/firewal

* Authentication: `$ gcloud auth login` rerun this in Google Cloud Notebook terminal when you are logged out and need the credential again.

### Installation

Download and install the latest (preview) version of the Vertex SDK for Python.

In [3]:
# ! pip install -U git+https://github.com/googleapis/python-aiplatform.git@main-test --user

Install the `h5py` to prepare sample dataset, and the `grpcio-tools` for querying against the index. 

In [4]:
! pip install -U grpcio-tools --user
# ! pip install -U h5py --user

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [5]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager).

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API, and Service Networking API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,servicenetworking.googleapis.com).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [6]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  rec-ai-demo-326116


### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with Vertex AI.

In [7]:
BUCKET_NAME = f"gs://{BUCKET}/bqml/item_embeddings"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

In [8]:
# from datetime import datetime

# TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

# if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == f"gs://{BUCKET}":
#     BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [9]:
# ! gsutil mb -l $REGION $BUCKET_NAME

CommandException: The mb command requires a URL that specifies a bucket.
"gs://rec_bq_jsw/bqml/item_embeddings" is not valid.


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
# ! gsutil ls -al $BUCKET_NAME

### Import libraries and define constants

Import the Vertex AI (unified) client library into your Python environment. 


In [13]:
import time

import grpc
from google.cloud import aiplatform_v1beta1
from google.protobuf import struct_pb2

In [14]:
REGION = "us-central1"
ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
NETWORK_NAME = "default"  # @param {type:"string"}


AUTH_TOKEN = !gcloud auth print-access-token
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

PARENT = "projects/{}/locations/{}".format(PROJECT_ID, REGION)

print("ENDPOINT: {}".format(ENDPOINT))
print("PROJECT_ID: {}".format(PROJECT_ID))
print("REGION: {}".format(REGION))

!gcloud config set project {PROJECT_ID}
!gcloud config set ai_platform/region {REGION}

ENDPOINT: us-central1-aiplatform.googleapis.com
PROJECT_ID: rec-ai-demo-326116
REGION: us-central1
Updated property [core/project].
Updated property [ai_platform/region].


## Prepare the Data

This data is generated from the 00-05 notebooks leveraging the css-retail dataset in BQ
You can find the example on [`02_export_bqml_mf_embeddings.ipynb`](02_export_bqml_mf_embeddings.ipynb) (linked)

The output csv from 02 can be found here

In [20]:
! gsutil ls {BUCKET_NAME}

gs://rec_bq_jsw/bqml/item_embeddings/embeddings-00000-of-00001.csv


## Create Indexes


### Create ANN Index (for Production Usage)

In [21]:
index_client = aiplatform_v1beta1.IndexServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)

In [22]:
DIMENSIONS = 50
DISPLAY_NAME = "retail_demo_matching_engine"
DISPLAY_NAME_BRUTE_FORCE = DISPLAY_NAME + "_brute_force"

Create the ANN index configuration:

Please read the documentation to understand the various configuration parameters that can be used to tune the index


In [23]:
treeAhConfig = struct_pb2.Struct(
    fields={
        "leafNodeEmbeddingCount": struct_pb2.Value(number_value=500),
        "leafNodesToSearchPercent": struct_pb2.Value(number_value=7),
    }
)

algorithmConfig = struct_pb2.Struct(
    fields={"treeAhConfig": struct_pb2.Value(struct_value=treeAhConfig)}
)

config = struct_pb2.Struct(
    fields={
        "dimensions": struct_pb2.Value(number_value=DIMENSIONS),
        "approximateNeighborsCount": struct_pb2.Value(number_value=150),
        "distanceMeasureType": struct_pb2.Value(string_value="DOT_PRODUCT_DISTANCE"),
        "algorithmConfig": struct_pb2.Value(struct_value=algorithmConfig),
    }
)

metadata = struct_pb2.Struct(
    fields={
        "config": struct_pb2.Value(struct_value=config),
        "contentsDeltaUri": struct_pb2.Value(string_value=BUCKET_NAME),
    }
)

ann_index = {
    "display_name": DISPLAY_NAME,
    "description": "Retail 50 Index",
    "metadata": struct_pb2.Value(struct_value=metadata),
}

In [24]:
ann_index = index_client.create_index(parent=PARENT, index=ann_index)

In [25]:
# Poll the operation until it's done successfullly.
# This will take ~50 min.

while True:
    if ann_index.done():
        break
    print("Poll the operation to create index...")
    time.sleep(60)

Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the ope

In [32]:
INDEX_RESOURCE_NAME = ann_index.result().name
INDEX_RESOURCE_NAME

'projects/733956866731/locations/us-central1/indexes/6094812855087923200'

### Create Brute Force Index (for Ground Truth)

The brute force index uses a naive brute force method to find the nearest neighbors. This method is not fast or efficient. Hence brute force indices are not recommended for production usage. They are to be used to find the "ground truth" set of neighbors, so that the "ground truth" set can be used to measure recall of the indices being tuned for production usage. To ensure an apples to apples comparison, the `distanceMeasureType` and `featureNormType`, `dimensions` of the brute force index should match those of the production indices being tuned.

Create the brute force index configuration:

In [ ]:
from google.protobuf import *

algorithmConfig = struct_pb2.Struct(
    fields={"bruteForceConfig": struct_pb2.Value(struct_value=struct_pb2.Struct())}
)

config = struct_pb2.Struct(
    fields={
        "dimensions": struct_pb2.Value(number_value=DIMENSIONS),
        "approximateNeighborsCount": struct_pb2.Value(number_value=150),
        "distanceMeasureType": struct_pb2.Value(string_value="DOT_PRODUCT_DISTANCE"),
        "algorithmConfig": struct_pb2.Value(struct_value=algorithmConfig),
    }
)

metadata = struct_pb2.Struct(
    fields={
        "config": struct_pb2.Value(struct_value=config),
        "contentsDeltaUri": struct_pb2.Value(string_value=BUCKET_NAME),
    }
)

brute_force_index = {
    "display_name": DISPLAY_NAME_BRUTE_FORCE,
    "description": "Glove 100 index (brute force)",
    "metadata": struct_pb2.Value(struct_value=metadata),
}

In [ ]:
brute_force_index = index_client.create_index(parent=PARENT, index=brute_force_index)

In [31]:
# Poll the operation until it's done successfullly.
# This will take ~45 min.

while True:
    if brute_force_index.done():
        break
    print("Poll the operation to create index...")
    time.sleep(60)

Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...


In [32]:
INDEX_BRUTE_FORCE_RESOURCE_NAME = brute_force_index.result().name
INDEX_BRUTE_FORCE_RESOURCE_NAME

'projects/733956866731/locations/us-central1/indexes/3552530865437278208'

## Update Indexes

Create incremental data file.


In [55]:
INC_BUCKET = "gs://rec_bq_jsw/bqml/incremental"

In [47]:
zero_emb_line = [0] + [0.0]*(DIMENSIONS) # +1 for ID = 0

with open("prod_emb_incremental.csv", "w") as f:
    f.write(
        ','.join([str(i) for i in zero_emb_line]) + "\n"
    )

Copy the incremental data file to a new subdirectory.


In [66]:
! gsutil cp prod_emb_incremental.csv {INC_BUCKET}/prod_emb_incremental.csv

Copying file://prod_emb_incremental.csv [Content-Type=text/csv]...
/ [1 files][  208.0 B/  208.0 B]                                                
Operation completed over 1 objects/208.0 B.                                      


Create update index request


In [67]:
metadata = struct_pb2.Struct(
    fields={
        "contentsDeltaUri": struct_pb2.Value(string_value=INC_BUCKET),
        "isCompleteOverwrite": struct_pb2.Value(bool_value=True)
    }
)

ann_index = {
    "name": INDEX_RESOURCE_NAME,
    "display_name": DISPLAY_NAME,
    "description": "Retail 50 Index",
    "metadata": struct_pb2.Value(struct_value=metadata),
}

In [68]:
ann_index = index_client.update_index(index=ann_index)

In [69]:
# Poll the operation until it's done successfullly.
# This will take ~45 min.

while True:
    if ann_index.done():
        break
    print("Poll the operation to update index...")
    time.sleep(60)

Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the operation to update index...
Poll the ope

In [70]:
INDEX_RESOURCE_NAME = ann_index.result().name
INDEX_RESOURCE_NAME

'projects/733956866731/locations/us-central1/indexes/4677304872373059584'

## Create an IndexEndpoint with VPC Network

In [39]:
index_endpoint_client = aiplatform_v1beta1.IndexEndpointServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)

In [72]:
VPC_NETWORK_NAME = "projects/{}/global/networks/{}".format(PROJECT_NUMBER, NETWORK_NAME)
VPC_NETWORK_NAME

'projects/733956866731/global/networks/default'

In [73]:
index_endpoint = {
    "display_name": "index_endpoint_for_demo",
    "network": VPC_NETWORK_NAME,
}

In [74]:
r = index_endpoint_client.create_index_endpoint(
    parent=PARENT, index_endpoint=index_endpoint
)

In [75]:
r.result()

name: "projects/733956866731/locations/us-central1/indexEndpoints/3027017083918483456"

In [76]:
INDEX_ENDPOINT_NAME = r.result().name
INDEX_ENDPOINT_NAME

'projects/733956866731/locations/us-central1/indexEndpoints/3027017083918483456'

## Deploy Indexes

### Deploy ANN Index

In [36]:
DEPLOYED_INDEX_ID = "ann_prod50_deployed"

In [37]:
deploy_ann_index = {
    "id": DEPLOYED_INDEX_ID,
    "display_name": DEPLOYED_INDEX_ID,
    "index": INDEX_RESOURCE_NAME,
}

In [38]:
r = index_endpoint_client.deploy_index(
    index_endpoint=INDEX_ENDPOINT_NAME, deployed_index=deploy_ann_index
)

NameError: name 'index_endpoint_client' is not defined

In [80]:
# Poll the operation until it's done successfullly.

while True:
    if r.done():
        break
    print("Poll the operation to deploy index...")
    time.sleep(60)

Poll the operation to deploy index...
Poll the operation to deploy index...
Poll the operation to deploy index...
Poll the operation to deploy index...


In [81]:
r.result()

deployed_index {
  id: "ann_prod50_deployed"
}

### Deploy Brute Force Index

In [82]:
DEPLOYED_BRUTE_FORCE_INDEX_ID = "prod50_brute_force_deployed"

In [83]:
deploy_brute_force_index = {
    "id": DEPLOYED_BRUTE_FORCE_INDEX_ID,
    "display_name": DEPLOYED_BRUTE_FORCE_INDEX_ID,
    "index": INDEX_BRUTE_FORCE_RESOURCE_NAME,
}

In [84]:
r = index_endpoint_client.deploy_index(
    index_endpoint=INDEX_ENDPOINT_NAME, deployed_index=deploy_brute_force_index
)

In [85]:
# Poll the operation until it's done successfullly.

while True:
    if r.done():
        break
    print("Poll the operation to deploy index...")
    time.sleep(60)

Poll the operation to deploy index...
Poll the operation to deploy index...
Poll the operation to deploy index...
Poll the operation to deploy index...


In [86]:
r.result()

deployed_index {
  id: "prod50_brute_force_deployed"
}

## Create Online Queries

After you built your indexes, you may query against the deployed index through the online querying gRPC API (Match service) within the virtual machine instances from the same region (for example 'us-central1' in this tutorial).  

The way a client uses this gRPC API is by folowing steps:

* Write `match_service.proto` locally
* Clone the repository that contains the dependencies of match_service.proto in the Terminal:

`$ mkdir third_party && cd third_party`

`$ git clone https://github.com/googleapis/googleapis.git`

* Compile the protocal buffer (see below)
* Obtain the index endpoint
* Use a code-generated stub to make the call, passing the parameter values

In [31]:
%%bash

mkdir third_party && cd third_party
git clone https://github.com/googleapis/googleapis.git

mkdir: cannot create directory ‘third_party’: File exists
Cloning into 'googleapis'...


In [33]:
%%writefile match_service.proto

syntax = "proto3";

package google.cloud.aiplatform.container.v1beta1;

import "google/rpc/status.proto";

// MatchService is a Google managed service for efficient vector similarity
// search at scale.
service MatchService {
  // Returns the nearest neighbors for the query. If it is a sharded
  // deployment, calls the other shards and aggregates the responses.
  rpc Match(MatchRequest) returns (MatchResponse) {}

  // Returns the nearest neighbors for batch queries. If it is a sharded
  // deployment, calls the other shards and aggregates the responses.
  rpc BatchMatch(BatchMatchRequest) returns (BatchMatchResponse) {}
}

// Parameters for a match query.
message MatchRequest {
  // The ID of the DeploydIndex that will serve the request.
  // This MatchRequest is sent to a specific IndexEndpoint of the Control API,
  // as per the IndexEndpoint.network. That IndexEndpoint also has
  // IndexEndpoint.deployed_indexes, and each such index has an
  // DeployedIndex.id field.
  // The value of the field below must equal one of the DeployedIndex.id
  // fields of the IndexEndpoint that is being called for this request.
  string deployed_index_id = 1;

  // The embedding values.
  repeated float float_val = 2;

  // The number of nearest neighbors to be retrieved from database for
  // each query. If not set, will use the default from
  // the service configuration.
  int32 num_neighbors = 3;

  // The list of restricts.
  repeated Namespace restricts = 4;

  // Crowding is a constraint on a neighbor list produced by nearest neighbor
  // search requiring that no more than some value k' of the k neighbors
  // returned have the same value of crowding_attribute.
  // It's used for improving result diversity.
  // This field is the maximum number of matches with the same crowding tag.
  int32 per_crowding_attribute_num_neighbors = 5;

  // The number of neighbors to find via approximate search before
  // exact reordering is performed. If not set, the default value from scam
  // config is used; if set, this value must be > 0.
  int32 approx_num_neighbors = 6;

  // The fraction of the number of leaves to search, set at query time allows
  // user to tune search performance. This value increase result in both search
  // accuracy and latency increase. The value should be between 0.0 and 1.0. If
  // not set or set to 0.0, query uses the default value specified in
  // NearestNeighborSearchConfig.TreeAHConfig.leaf_nodes_to_search_percent.
  int32 leaf_nodes_to_search_percent_override = 7;
}

// Response of a match query.
message MatchResponse {
  message Neighbor {
    // The ids of the matches.
    string id = 1;

    // The distances of the matches.
    double distance = 2;
  }
  // All its neighbors.
  repeated Neighbor neighbor = 1;
}

// Parameters for a batch match query.
message BatchMatchRequest {
  // Batched requests against one index.
  message BatchMatchRequestPerIndex {
    // The ID of the DeploydIndex that will serve the request.
    string deployed_index_id = 1;

    // The requests against the index identified by the above deployed_index_id.
    repeated MatchRequest requests = 2;

    // Selects the optimal batch size to use for low-level batching. Queries
    // within each low level batch are executed sequentially while low level
    // batches are executed in parallel.
    // This field is optional, defaults to 0 if not set. A non-positive number
    // disables low level batching, i.e. all queries are executed sequentially.
    int32 low_level_batch_size = 3;
  }

  // The batch requests grouped by indexes.
  repeated BatchMatchRequestPerIndex requests = 1;
}

// Response of a batch match query.
message BatchMatchResponse {
  // Batched responses for one index.
  message BatchMatchResponsePerIndex {
    // The ID of the DeployedIndex that produced the responses.
    string deployed_index_id = 1;

    // The match responses produced by the index identified by the above
    // deployed_index_id. This field is set only when the query against that
    // index succeed.
    repeated MatchResponse responses = 2;

    // The status of response for the batch query identified by the above
    // deployed_index_id.
    google.rpc.Status status = 3;
  }

  // The batched responses grouped by indexes.
  repeated BatchMatchResponsePerIndex responses = 1;
}

// Namespace specifies the rules for determining the datapoints that are
// eligible for each matching query, overall query is an AND across namespaces.
message Namespace {
  // The string name of the namespace that this proto is specifying,
  // such as "color", "shape", "geo", or "tags".
  string name = 1;

  // The allowed tokens in the namespace.
  repeated string allow_tokens = 2;

  // The denied tokens in the namespace.
  // The denied tokens have exactly the same format as the token fields, but
  // represents a negation. When a token is denied, then matches will be
  // excluded whenever the other datapoint has that token.
  //
  // For example, if a query specifies {color: red, blue, !purple}, then that
  // query will match datapoints that are red or blue, but if those points are
  // also purple, then they will be excluded even if they are red/blue.
  repeated string deny_tokens = 3;
}

Overwriting match_service.proto


Compile the protocol buffer, and then `match_service_pb2.py` and `match_service_pb2_grpc.py` are generated.

In [34]:
! python -m grpc_tools.protoc -I=. --proto_path=third_party/googleapis --python_out=. --grpc_python_out=. match_service.proto

Obtain the Private Endpoint: 

In [40]:
DEPLOYED_INDEX_SERVER_IP = (
    list(index_endpoint_client.list_index_endpoints(parent=PARENT))[0]
    .deployed_indexes[0]
    .private_endpoints.match_grpc_address
)
DEPLOYED_INDEX_SERVER_IP

'10.120.0.5'

Test your query:

In [41]:
import match_service_pb2
import match_service_pb2_grpc

channel = grpc.insecure_channel("{}:10000".format(DEPLOYED_INDEX_SERVER_IP))
stub = match_service_pb2_grpc.MatchServiceStub(channel)

In [43]:
import numpy as np
# Test query
np.random.seed(12341455)
query = list(np.random.rand(DIMENSIONS)*100)

request = match_service_pb2.MatchRequest()
request.deployed_index_id = DEPLOYED_INDEX_ID
for val in query:
    request.float_val.append(val)

response = stub.Match(request)
response

neighbor {
  id: "9999999"
}

### Batch Query

You can run multiple queries in a single RPC call using the BatchMatch API:

In [107]:
def get_request(embedding, deployed_index_id):
    request = match_service_pb2.MatchRequest(num_neighbors=k)
    request.deployed_index_id = deployed_index_id
    for val in embedding:
        request.float_val.append(val)
    return request

In [28]:
from datetime import datetime
import time
import numpy as np

# Test query
n_examples = 10
k = 10
queries = [list(np.random.rand(DIMENSIONS)) for _ in range(n_examples)]

batch_request = match_service_pb2.BatchMatchRequest()
batch_request_ann = match_service_pb2.BatchMatchRequest.BatchMatchRequestPerIndex()
batch_request_brute_force = (
    match_service_pb2.BatchMatchRequest.BatchMatchRequestPerIndex()
)
batch_request_ann.deployed_index_id = DEPLOYED_INDEX_ID
batch_request_brute_force.deployed_index_id = DEPLOYED_BRUTE_FORCE_INDEX_ID
for query in queries:
    batch_request_ann.requests.append(get_request(query, DEPLOYED_INDEX_ID))
    batch_request_brute_force.requests.append(
        get_request(query, DEPLOYED_BRUTE_FORCE_INDEX_ID)
    )
batch_request.requests.append(batch_request_ann)
batch_request.requests.append(batch_request_brute_force)

response = stub.BatchMatch(batch_request)
response

NameError: name 'match_service_pb2' is not defined

### Compute Recall

Use deployed brute force Index as the ground truth to calculate the recall of ANN Index:

In [112]:
def get_neighbors(embedding, deployed_index_id):
    request = match_service_pb2.MatchRequest(num_neighbors=k)
    request.deployed_index_id = deployed_index_id
    for val in embedding:
        request.float_val.append(val)
    response = stub.Match(request)
    return [int(n.id) for n in response.neighbor]

In [114]:
# This will take 5-10 min

test=queries #set test to the 
recall = sum(
    [
        len(
            set(get_neighbors(test[i], DEPLOYED_BRUTE_FORCE_INDEX_ID)).intersection(
                set(get_neighbors(test[i], DEPLOYED_INDEX_ID))
            )
        )
        for i in range(len(test))
    ]
) / (1.0 * len(test) * k)

print("Recall: {}".format(recall))

Recall: 0.0


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.
You can also manually delete resources that you created by running the following code.

In [47]:
index_client.delete_index(name=INDEX_RESOURCE_NAME)
index_client.delete_index(name=INDEX_BRUTE_FORCE_RESOURCE_NAME)

NotFound: 404 Index `projects/733956866731/locations/us-central1/indexes/6094812855087923200` is not found.

In [44]:
index_endpoint_client.delete_index_endpoint(name=INDEX_ENDPOINT_NAME)

NameError: name 'INDEX_ENDPOINT_NAME' is not defined